In [ ]:
#OLD CFTOC CODE, ALWAYS KEEPS SAYING INFEASIBLE
#Solves a CFTOC problem
def solve_cftoc(nq, nu, P, Q, R, N, q0, qL, qU, uL, uU, bf, Af, qref, uref):
  """
  Solves a CFTOC problem
    q_ref, u_ref: reference trajectory, should be of length N, of size nq and nu respectively
  
  """
  model = pyo.ConcreteModel()
  model.N = N                   # number of steps
  model.nq = nq                 # number of states
  model.nu = nu                 # number of inputs
  model.nf = np.size(Af, 0)     # number of terminal states

  # length of finite optimization problem:
  model.tIDX = pyo.Set(initialize= range(model.N+1), ordered=True)
  model.qIDX = pyo.Set(initialize= range(model.nq), ordered=True)
  model.uIDX = pyo.Set(initialize= range(model.nu), ordered=True)
  model.nfIDX = pyo.Set(initialize= range(model.nf), ordered=True)

  model.q = pyo.Var(model.qIDX, model.tIDX)
  model.u = pyo.Var(model.uIDX, model.tIDX)

  model.P = P; model.Q = Q; model.R = R
  model.Af = Af; model.Bf = bf

  
  #This function returns the instantaneous value of the i-th element of the state vector, at time t
  #This is the continuous time dynamics, q_dot = f(x,u)
  def sysDyn_CT_qdot(model, i, t):
    global r_cm, L_cm, Ts
    psi_k = model.q[2,t]

    B_k = np.array([[r_cm/2*pyo.cos(psi_k), r_cm/2*pyo.cos(psi_k)],
              [r_cm/2*pyo.sin(psi_k), r_cm/2*pyo.sin(psi_k)],
              [-r_cm/L_cm, r_cm/L_cm],
              [1, 0],
              [0, 1]])
    
    return (B_k[i,0] * model.u[0,t] + B_k[i,1] * model.u[1,t])

  #This function returns the k+1 value of the i-th element of the state vector at time t
  #This is the discrete time dynamics, q_{k+1} = q_k + Ts * f(q_k,u_k), where q_dot = f(q,u)
  def sysDyn_disc_kplus1(model, i, t):
    global r_cm, L_cm, Ts
    return model.q[i,t] + Ts * sysDyn_CT_qdot(model, i, t)

  def dynamics_constraints(model, i, t):
    # B = B_k(model, t)
    # return model.q[i, t+1] - (model.q[i,t] + sum(B[i,j]*model.u[j,t] for j in model.uIDX)) == 0.0 if t < model.N else pyo.Constraint.Skip
    return model.q[i, t+1] - sysDyn_disc_kplus1(model, i, t) == 0.0 if t < model.N else pyo.Constraint.Skip
  
  def final_constraints(model, i):
    return sum(model.Af[i,j]*model.q[j,model.N] for j in model.qIDX) <= model.Bf[i]

  model.dynamics_constraints = pyo.Constraint(model.qIDX, model.tIDX, rule=dynamics_constraints)
  model.init_constraints = pyo.Constraint(model.qIDX, rule = lambda model, i: model.q[i,0] == q0[i])
  model.terminal_constraints = pyo.Constraint(model.nfIDX, rule = final_constraints)

  # State and input constraints
  model.qL_constraints = pyo.Constraint(model.qIDX, model.tIDX, rule = lambda model, i, t: model.q[i,t] >= qL[i])
  model.qU_constraints = pyo.Constraint(model.qIDX, model.tIDX, rule = lambda model, i, t: model.q[i,t] <= qU[i])
  model.uL_constraints = pyo.Constraint(model.uIDX, model.tIDX, rule = lambda model, i, t: model.u[i,t] >= uL[i])
  model.uU_constraints = pyo.Constraint(model.uIDX, model.tIDX, rule = lambda model, i, t: model.u[i,t] <= uU[i])

  def objective_rule(model):
    costQ = 0.0
    costU = 0.0
    costTerminal = 0.0

    # We need to use qref/uref[t-1] because we don't use the control at time 0
    for t in model.tIDX:
      for i in model.qIDX:
        for j in model.qIDX:
          if t < model.N:
            costQ += (model.q[i,t]-qref[i,t]) * model.Q[i,j] * (model.q[j,t]-qref[j,t])
    for t in model.tIDX:
      for i in model.uIDX:
        for j in model.uIDX:
          if t < model.N:
            costU += (model.u[i,t]-uref[i,t]) * model.R[i,j] * (model.u[j,t]-uref[i,t])
    for i in model.qIDX:
      for j in model.qIDX:
        costTerminal += (model.q[i,model.N]-qref[i,model.N]) * model.P[i,j] * (model.q[j,model.N]-qref[i,model.N])
        
    return costQ + costU + costTerminal

  model.cost = pyo.Objective(rule = objective_rule, sense = pyo.minimize)

  solver = pyo.SolverFactory('ipopt')
  results = solver.solve(model)

  feas = str(results.solver.termination_condition) == "optimal"
  xOpt = np.asarray([[model.q[i,t]() for i in model.qIDX] for t in model.tIDX]).T
  uOpt = np.asarray([model.u[:,t]() for t in model.tIDX]).T
  JOpt = model.cost()

  return [feas, xOpt, uOpt, JOpt, model, results]


In [ ]:
def batch_trajectory(N):
    nq = 5
    nu = 2
    P = np.eye(5)
    # Q = np.diag([1, 2, 2, 3, 3])
    Q = np.eye(5)
    R = np.eye(5)

    q0 = np.array([0, 0, 0, np.pi/2, 0])

    # State constraints
    qL = np.array([-100, -100, -np.inf, -np.inf, -np.inf])
    qU = np.array([100, 100, np.inf, np.inf, np.inf])
    uL = np.array([-1, -1])* 2*np.pi             #Max wheel rotation speed in rad/s
    uU = np.array([1, 1]) * 2*np.pi

    # Final state constraints
    phi_final = 0       #THIS IS NOT DEFINED RIGHT, is a polytope centered at 0 now with Aq <= b, need affine polytope
    x_final_bound = 0.5
    y_final_bound = 0.5
    psi_final_bound = np.deg2rad(1)
    phi_final_bound = np.deg2rad(1)
    Af = np.vstack((np.eye(5), -np.eye(5)))
    bf = np.array([x_final_bound, y_final_bound, psi_final_bound, phi_final_bound, phi_final_bound,       # upper bounds
                   x_final_bound, y_final_bound, psi_final_bound, phi_final_bound, phi_final_bound])      # lower bounds
    qref = np.zeros((5, 1001))
    uref = np.zeros((2, 1001))
    
    [feas, qOpt, uOpt, JOpt, model, results] = solve_cftoc(nq, nu, P, Q, R, N, q0, qL, qU, uL, uU, bf, Af, qref, uref)
    print(f"Optimal cost = {JOpt}")
    # model.pprint()

    # Check the KKT conditions

    return qOpt, uOpt, N, feas
